<a href="https://colab.research.google.com/github/shriver42/ESAA/blob/%EC%97%B0%EC%8A%B5-%EB%AC%B8%EC%A0%9C/YB_5_3_%EC%84%B8%EC%85%98_%EB%AC%B8%EC%A0%9C_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%B6%84%EB%A5%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

1. 필요한 라이브러리를 가져옵니다.

In [176]:
import pandas as pd
import lightgbm as lgbm

2. 데이터를 불러옵니다.

In [177]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [178]:
train = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/test.csv')
smpl_sub = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/sample_submission.csv')

In [179]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [180]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


3. 변수를 변환합니다.

In [181]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.
from sklearn.preprocessing import LabelEncoder

print(train['type'].unique())


#LabelEncoder를 객체로 생성
encoder = LabelEncoder()
encoder.fit(train['type'])
train['type'] = encoder.transform(train['type'])
encoder.fit(test['type'])
test['type'] = encoder.transform(test['type'])

['white' 'red']


In [182]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1


In [183]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1


In [184]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.

train_x = train.drop(train.columns[[0,1]], axis=1, inplace=False)
train_y = train[["quality"]]
test_x = test.drop('index', axis=1, inplace=False)

In [185]:
train_x.shape, train_y.shape, test_x.shape

((5497, 12), (5497, 1), (1000, 12))

In [186]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1


In [187]:
train_y.head()

,quality
0,5
1,5
2,5
3,6
4,6


In [188]:
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1


4. LightGBM 모델을 생성 및 훈련합니다.

In [189]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier()
lgbm_model.fit(train_x, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1538
[LightGBM] [Info] Number of data points in the train set: 5497, number of used features: 12
[LightGBM] [Info] Start training from score -5.353861
[LightGBM] [Info] Start training from score -3.386211
[LightGBM] [Info] Start training from score -1.123105
[LightGBM] [Info] Start training from score -0.822089
[LightGBM] [Info] Start training from score -1.783246
[LightGBM] [Info] Start training from score -3.588077
[LightGBM] [Info] Start training from score -7.002520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

LGBMClassifier()

5. 생성한 모델로 예측 데이터를 생성합니다.

In [190]:
y_pred = lgbm_model.predict(test_x)

6. 제출파일을 생성합니다.

In [191]:
smpl_sub.head()

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [192]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
smpl_sub['quality'] = y_pred

In [193]:
smpl_sub.head()

,index,quality
0,0,5
1,1,5
2,2,6
3,3,5
4,4,6


In [194]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [195]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2024-05-03 11:03:31--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2024-05-03 11:03:31--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.101.139, 142.250.101.101, 142.250.101.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2024-05-03 11:03:31--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)...

In [196]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [197]:
# 데이터 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5497 entries, 0 to 5496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quality               5497 non-null   int64  
 1   fixed acidity         5497 non-null   float64
 2   volatile acidity      5497 non-null   float64
 3   citric acid           5497 non-null   float64
 4   residual sugar        5497 non-null   float64
 5   chlorides             5497 non-null   float64
 6   free sulfur dioxide   5497 non-null   float64
 7   total sulfur dioxide  5497 non-null   float64
 8   density               5497 non-null   float64
 9   pH                    5497 non-null   float64
 10  sulphates             5497 non-null   float64
 11  alcohol               5497 non-null   float64
 12  type                  5497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 558.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries,

In [198]:
# 원핫 인코딩 (pd.get_dummies())
train_one = pd.get_dummies(data = train, columns = ['type'], prefix = ['type'])
test_one = pd.get_dummies(data = test, columns = ['type'], prefix = ['type'])

encoder = LabelEncoder()
train_one['quality'] = encoder.fit_transform(train_one['quality'])

# 5-> 2로 변환
train_one.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,2,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,False,True
1,2,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,True,False
2,2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,False,True
3,3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,False,True
4,3,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,False,True


In [199]:
# 모델 정의
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()

In [200]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X = train_one.drop('quality', axis=1, inplace=False)
y = train_one['quality']


# fit 메소드를 이용해 모델 학습

xgb_clf.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [201]:
# predict 메소드와 원핫 인코딩한 test 데이터를 이용해 품질 예측
xgb_pred = xgb_clf.predict(test_one)

In [202]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
submission = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/sample_submission.csv')
submission['quality'] = xgb_pred +3     #앞에서 인코딩을 통해 5->2로 변환함
submission.head()

,index,quality
0,0,5
1,1,5
2,2,5
3,3,5
4,4,6


In [203]:
# 정답파일 내보내기
submission.to_csv('xgb_pred.csv',index = False)

# 3. 보팅

In [204]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2024-05-03 11:03:34--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2024-05-03 11:03:34--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.101.139, 142.250.101.101, 142.250.101.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2024-05-03 11:03:34--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)...

In [205]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

train = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/와인품질분류/data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [206]:
# 데이터 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5497 entries, 0 to 5496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quality               5497 non-null   int64  
 1   fixed acidity         5497 non-null   float64
 2   volatile acidity      5497 non-null   float64
 3   citric acid           5497 non-null   float64
 4   residual sugar        5497 non-null   float64
 5   chlorides             5497 non-null   float64
 6   free sulfur dioxide   5497 non-null   float64
 7   total sulfur dioxide  5497 non-null   float64
 8   density               5497 non-null   float64
 9   pH                    5497 non-null   float64
 10  sulphates             5497 non-null   float64
 11  alcohol               5497 non-null   float64
 12  type                  5497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 558.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries,

In [207]:
# 원핫 인코딩 (pd.get_dummies())
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [208]:
# 모델 정의 (LightGBM, XGBoost, Random Forest)
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

lgbm = LGBMClassifier()
xgb = XGBClassifier()
rf = RandomForestClassifier()

# VotingClassifier 정의
vo_clf = VotingClassifier(estimators=[('lgbm', lgbm), ('xgb', xgb), ('rf', rf)], voting='soft')

In [209]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X = train_ohe.drop('quality', axis=1, inplace=False)
y = train_ohe[['quality']]

# fit 메소드를 이용해 모델 학습

vo_clf.fit(X,y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1540
[LightGBM] [Info] Number of data points in the train set: 5497, number of used features: 13
[LightGBM] [Info] Start training from score -5.353861
[LightGBM] [Info] Start training from score -3.386211
[LightGBM] [Info] Start training from score -1.123105
[LightGBM] [Info] Start training from score -0.822089
[LightGBM] [Info] Start training from score -1.783246
[LightGBM] [Info] Start training from score -3.588077
[LightGBM] [Info] Start training from score -7.002520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

VotingClassifier(estimators=[('lgbm', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [210]:
# predict 메소드와 원핫 인코딩한 테스트 데이터를 이용해 품질 예측
pred = vo_clf.predict(test_ohe)
print(pred)

[5 5 6 5 6 6 6 6 6 7 6 6 7 6 5 5 6 6 5 5 6 6 5 7 6 5 5 5 5 5 7 6 5 6 5 6 5
 5 6 6 6 5 5 6 6 6 5 6 4 6 5 5 6 4 5 6 6 5 5 6 5 6 6 4 6 6 5 5 6 5 5 5 5 6
 6 5 6 6 6 7 6 6 5 4 6 6 5 5 6 6 6 6 6 6 6 7 6 7 6 6 5 6 6 5 5 6 6 6 7 5 6
 6 6 5 5 6 6 7 7 6 6 5 6 7 5 5 5 6 6 7 6 5 6 5 5 6 6 7 5 5 5 6 5 5 5 5 7 6
 7 6 5 8 6 5 6 5 5 5 6 5 5 6 6 7 6 5 6 6 5 6 5 5 6 5 6 6 6 5 6 5 6 6 7 7 5
 6 6 6 5 6 6 6 6 6 6 6 7 5 5 5 6 5 6 5 5 5 6 6 6 6 7 5 5 6 6 6 6 6 6 7 6 5
 6 6 7 6 6 5 6 5 6 7 7 6 6 6 6 6 6 7 6 6 6 6 7 6 6 7 5 6 5 6 7 7 6 6 6 7 7
 6 7 5 5 7 5 5 6 5 6 6 6 5 5 5 6 5 7 7 5 6 6 6 6 6 7 7 6 6 6 6 5 5 5 6 5 7
 8 5 5 7 6 6 5 5 6 7 5 7 5 5 6 5 6 5 6 6 5 5 6 6 5 5 6 5 4 5 5 7 6 6 5 6 7
 5 5 7 5 6 6 6 5 6 7 6 5 6 5 6 5 6 5 6 5 6 6 5 6 6 6 6 5 6 6 7 7 6 5 5 7 6
 6 6 5 5 6 5 5 6 6 6 7 6 7 6 5 6 6 5 6 6 7 5 7 6 5 6 6 7 7 5 7 6 6 6 7 5 6
 6 6 6 6 6 6 5 6 6 6 6 5 6 7 6 5 6 6 6 6 6 5 6 7 5 5 6 6 4 6 5 5 5 5 7 6 5
 6 6 5 5 5 6 6 7 5 6 6 6 7 5 6 5 5 6 5 6 5 6 6 6 6 5 5 5 5 6 5 6 6 8 7 6 6
 7 6 5 8 5 5 6 5 6 7 6 5 